In [444]:
import pandas as pd


In [445]:
df = pd.read_csv(rf"C:\D_Drive\ASM\experiment\tmp\semanticType__weapon-lod.csv")

In [446]:
df

,data_source,column_name,correct_type,candidate_types
0,alaskaslist.csv,Price1219,Offer;price,[Offer;price;0.9999426868422568\tCreativeWork;...
1,alaskaslist.csv,Alaska0011,Offer;name,[Offer;name;0.47152383671818865\tOffer;descrip...
2,alaskaslist.csv,Ad1096,Offer;identifier,[]
3,alaskaslist.csv,Updated1527,Offer;datePosted,[Offer;datePosted;0.8902875080455296\tOffer;de...
4,alaskaslist.csv,Copyright1795,CreativeWork;copyrightYear,[Offer;price;0.5\tCreativeWork;copyrightYear;0...
...,...,...,...,...
144,theoutdoorstrader.csv,Price0855,Offer;price,[Offer;price;0.5031652685269817\tOffer;name;0....
145,theoutdoorstrader.csv,Accessories0378,Firearm;category,[Firearm;category;0.6010879908020876\tOffer;na...
146,theoutdoorstrader.csv,file_name,CreativeWork;name,[CreativeWork;name;1.0\t]
147,theoutdoorstrader.csv,trade1645,CreativeWork;copyrightYear,[CreativeWork;copyrightYear;0.9861111115716481...


In [447]:
df = df[df.candidate_types != "[]"]

In [448]:
df

,data_source,column_name,correct_type,candidate_types
0,alaskaslist.csv,Price1219,Offer;price,[Offer;price;0.9999426868422568\tCreativeWork;...
1,alaskaslist.csv,Alaska0011,Offer;name,[Offer;name;0.47152383671818865\tOffer;descrip...
3,alaskaslist.csv,Updated1527,Offer;datePosted,[Offer;datePosted;0.8902875080455296\tOffer;de...
4,alaskaslist.csv,Copyright1795,CreativeWork;copyrightYear,[Offer;price;0.5\tCreativeWork;copyrightYear;0...
5,alaskaslist.csv,file_name,CreativeWork;name,[CreativeWork;name;1.0\t]
...,...,...,...,...
144,theoutdoorstrader.csv,Price0855,Offer;price,[Offer;price;0.5031652685269817\tOffer;name;0....
145,theoutdoorstrader.csv,Accessories0378,Firearm;category,[Firearm;category;0.6010879908020876\tOffer;na...
146,theoutdoorstrader.csv,file_name,CreativeWork;name,[CreativeWork;name;1.0\t]
147,theoutdoorstrader.csv,trade1645,CreativeWork;copyrightYear,[CreativeWork;copyrightYear;0.9861111115716481...


In [449]:
class SemanticType:
    def __init__(self, typeStr:str):
        ls = typeStr.split(";")
        self.domain = ls[0]
        self.type = ls[1]
        if len(ls) == 2:
            self.score = 1.0
        else:
            self.score = float(ls[2])

    def __repr__(self):
        return ",".join([self.domain, self.type, str(self.score)])
        

class Column:
    def __init__(self, data_source:str, column_name:str, correct_type:str, candidate_types:str):
        self.data_source = data_source
        self.column_name = column_name
        self.correct_type = SemanticType(correct_type)
        self.candidate_types = self._get_candidate_types(candidate_types)
        if len(self.candidate_types) > 1:
            self.eta = self._get_eta()


    def _get_candidate_types(self, candidate_types:str):
        typeStr =  candidate_types.replace("[", "").replace("]", "").rstrip("\t")
        ls = typeStr.split("\t")
        return [SemanticType(ele) for ele in ls]

    def _get_eta(self):

        try:
            return self.candidate_types[0].score / self.candidate_types[1].score
        except ZeroDivisionError:
            return float("inf")

    



In [450]:
columns = [Column(col["data_source"], col["column_name"], col["correct_type"], col["candidate_types"]) for _, col in df.iterrows()]

In [451]:
mrr = 0.0
for col in columns:
    reciprocal_rank = 0
    for i, cType in enumerate(col.candidate_types):
        if cType.domain == col.correct_type.domain and cType.type == col.correct_type.type:
            reciprocal_rank = 1 / (i+1)
    mrr += reciprocal_rank

mrr /= len(columns)
mrr

0.8788968824940047

In [452]:
len(columns)

139

In [453]:
len([col for col in columns if col.candidate_types[0].score > 0.5])

114

In [454]:
def f(cols):
    res = []
    for col in cols:
        candidate_types = col.candidate_types
        for i, cType in enumerate(candidate_types):
            if cType.domain == col.correct_type.domain and cType.type == col.correct_type.type and i > 0:
                res.append(col)    
    return res


len(f(columns))


26

In [455]:
for i in range(1, 11):
    print(len(f([col for col in columns if (len(col.candidate_types) > 1) and col.eta > i])), end=" ")

20 9 7 7 6 6 6 6 6 6 

In [456]:
len(f([col for col in columns if col.candidate_types[0].score > 0.6]))

8

In [457]:
rs = []

for col in columns:
    candidate_types = col.candidate_types
    for i, cType in enumerate(candidate_types):
        if cType.domain == col.correct_type.domain and cType.type == col.correct_type.type:
            rs.append(i)
            # print(i)


print(rs.count(0), rs.count(1), rs.count(2), rs.count(3), len(columns) - len(rs))

110 21 5 0 3


In [458]:
'''
224 25 9 0 6
271 23 8 3 0
110 21 5 0 3
33 10 4 3 2 1 
34 7 2 1 0 0 
20 9 7 7 6 6 
'''

'\n224 25 9 0 6\n271 23 8 3 0\n110 21 5 0 3\n33 10 4 3 2 1 \n34 7 2 1 0 0 \n20 9 7 7 6 6 \n'